# 1. Create an iceberg table with AWS Glue Catalog in python

Using pyiceberg to create an iceberg dataset on S3;
* https://py.iceberg.apache.org/#more-details 

## Setup

In [1]:
!pip install --upgrade --no-warn-conflicts pip
# you may need to reload the kernel to take pydantic version update into account
!pip install -qU --no-warn-conflicts pydantic
!pip install -q --no-warn-conflicts "pyiceberg[s3fs,hive,glue,sql-sqlite]"

In [2]:
# restart kernel for packages to take effect
from IPython.core.display import HTML
display(HTML("<script>Jupyter.notebook.kernel.restart()</script>"))

In [3]:
import sagemaker

default_bucket = sagemaker.Session().default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Iceberg with Glue catalog

In [4]:
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o /tmp/yellow_tripdata_2023-01.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0   205M      0 --:--:-- --:--:-- --:--:--  206M


In [5]:
%%writefile ~/.pyiceberg.yaml
catalog:
  default:
    type: glue

Overwriting /home/sagemaker-user/.pyiceberg.yaml


In [6]:
from pyiceberg.catalog import load_catalog
from pyiceberg.table import Table
from pyiceberg.schema import Schema
from pyiceberg.types import DoubleType, StringType, TimestampType, NestedField
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import YearTransform, MonthTransform, DayTransform
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform

In [7]:
# Specify the Glue Catalog database name and URI
glue_database_name = "iceberg_glue_catalog"
catalog_name = "iceberg_catalog"  # replace with your catalog name
glue_catalog_uri = f"s3://{default_bucket}/{catalog_name}"
table_name = f"{glue_database_name}.taxi_dataset"

In [8]:
# Instantiate glue catalog
glue_catalog = load_catalog("glue", **{"type": "glue"})

In [9]:
import pyarrow.parquet as pq

df = pq.read_table("/tmp/yellow_tripdata_2023-01.parquet")

In [10]:
df.schema

VendorID: int64
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: double
trip_distance: double
RatecodeID: double
store_and_fwd_flag: string
PULocationID: int64
DOLocationID: int64
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2492

In [11]:
try:
    glue_catalog.create_namespace(glue_database_name)
    print(f"Namespace {glue_database_name} created")
except Exception as e:
    print(e)
    # table = catalog.load_table("default.taxi_dataset")


Database iceberg_glue_catalog already exists


In [12]:
try:
    table = glue_catalog.create_table(
        table_name,
        schema=df.schema,
        location=glue_catalog_uri
    )
    print(f"table {table_name} created")
except Exception as e:
    print(e)
    table = glue_catalog.load_table(table_name)


Table iceberg_glue_catalog.taxi_dataset already exists


In [13]:
table.overwrite(df)
len(table.scan().to_arrow())

3066766

You can list all namespaces using

```python
namespaces = glue_catalog.list_namespaces()
namespaces
```

Likewise, you can list all tables within a namespace as follows:

```python
for (namespace,) in namespaces:
    print(f"tables in namespace {namespace}")
    print(glue_catalog.list_tables(namespace))
    print()
```

Below we list the tables in the current namespace (i.e database name)

In [14]:
glue_catalog.list_tables(glue_database_name)

[('iceberg_glue_catalog', 'taxi_dataset')]